<a href="https://colab.research.google.com/github/wasanapelawaththa/Garbage-classification-model/blob/main/classXinit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
#Install and import required libraries
!pip install tensorflow keras scikit-learn

import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files

In [35]:
#Set dataset path
dataset_path = "/content/drive/MyDrive/data"

#Define parameters
img_size = (300, 300)
batch_size = 16
num_classes = 5

In [36]:
class_counts = {}
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        class_counts[class_name] = len(os.listdir(class_path))

print(class_counts)

{'plastic': 400, 'metal': 400, 'paper': 400, 'glass': 400, 'cardboard': 400}


In [38]:
#Data Preprocessing + Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 1600 images belonging to 5 classes.
Found 400 images belonging to 5 classes.


In [39]:
#Load Pretrained Model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
base_model.trainable = False

#Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

#Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Train base model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 572s 5s/step - accuracy: 0.7035 - loss: 0.9163 - val_accuracy: 0.8675 - val_loss: 0.3635
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 568s 6s/step - accuracy: 0.9040 - loss: 0.3173 - val_accuracy: 0.8750 - val_loss: 0.3446
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 531s 5s/step - accuracy: 0.9060 - loss: 0.2541 - val_accuracy: 0.8825 - val_loss: 0.3520
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 599s 6s/step - accuracy: 0.9051 - loss: 0.2458 - val_accuracy: 0.8625 - val_loss: 0.4455
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 515s 5s/step - accuracy: 0.9390 - loss: 0.1787 - val_accuracy: 0.8925 - val_loss: 0.3570
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 564s 6s/step - accuracy: 0.9400 - loss: 0.1667 - val_accuracy: 0.8725 - val_loss: 0.4287
Epoch 7/10
 20/100 ━━━━━━━━━━━━━━━━━━━━ 5:42 4s/step - accuracy: 0.9439 - loss: 0.1856

In [ ]:
#Plot Accuracy & Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(list(history.history['accuracy']) + list(history_fine.history['accuracy']), label='Train Accuracy')
plt.plot(list(history.history['val_accuracy']) + list(history_fine.history['val_accuracy']), label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(list(history.history['loss']) + list(history_fine.history['loss']), label='Train Loss')
plt.plot(list(history.history['val_loss']) + list(history_fine.history['val_loss']), label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Fine-tune last layers
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

In [ ]:
#Evaluation - Confusion Matrix & Classification Report
val_generator.reset()
Y_pred = model.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)

class_labels = list(train_generator.class_indices.keys())

print("\nClassification Report:\n")
print(classification_report(val_generator.classes, y_pred, target_names=class_labels))

cm = confusion_matrix(val_generator.classes, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
#Upload an image and predict
# Upload an image
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename
    img = image.load_img(img_path, target_size=img_size)  # resize
    img_array = image.img_to_array(img) / 255.0           # normalize
    img_array = np.expand_dims(img_array, axis=0)         # add batch dimension

    # Predict
    pred = model.predict(img_array)
    class_idx = np.argmax(pred)
    print(f"Predicted Class: {class_labels[class_idx]} - Confidence: {np.max(pred)*100:.2f}%")

In [ ]:
#Save Model
model.save("/content/initialtrain.h5")
print("Model saved at /content/initialtrain.h5")

In [ ]:
#Fine-tuning after initial training
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Unfreeze the last layers
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)
]

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks
)

loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy after fine-tuning: {acc*100:.2f}%")